In [1]:
from urllib.request import urlopen
from urllib.parse import urljoin
from urllib.parse import urlparse, parse_qs
import requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import time

In [132]:
engine = create_engine('sqlite:///nfl_fantasy.db')


In [130]:
df = pd.read_csv('FantasyDataExport.csv')

In [140]:
df['Player'].nunique()

56

In [146]:
df.groupby(['Week', 'Player']).sum()

Rk  Comp  Att  Pct  Yds  Yds/Att  TD  Int  \
Week Player                                                           
1    Aaron Rodgers         4    30   44   68  303        7   2    1   
     Alex Smith           12    20   26   77  211        8   2    0   
     Andrew Luck          26    23   45   51  309        7   1    3   
     Andy Dalton          30    22   37   60  221        6   0    1   
     Ben Roethlisberger   15    22   40   55  245        6   2    1   
     Blaine Gabbert        9    23   39   59  260        7   2    0   
     Brandon Weeden       39    12   35   34  118        3   0    4   
     Bruce Gradkowski     38     0    0    0    0        0   0    0   
     Cam Newton           21    23   33   70  303        9   1    2   
     Carson Palmer        11    32   46   70  297        7   1    0   
     Chase Daniel         37     0    0    0    0        0   0    0   
     Christian Ponder     27    20   27   74  270       10   0    0   
     Colin Kaepernick     35     0    0    0    0        0   0    0   
     Drew Brees            5    24   52   46  339        7   3    2   
     Eli Manning          20    21   32   66  213        7   1    0   
     Jake Locker          25    23   32   72  229        7   1    1   
     Jay Cutler           10    21   35   60  333       10   2    1   
     Joe Flacco            8    21   29   72  299       10   2    0   
     John Skelton         31    14   28   50  149        5   0    1   
     Josh Freeman         24    16   24   67  138        6   1    0   
     Kevin Kolb           29     6    8   75   66        8   1    0   
     Mark Sanchez          6    19   27   70  266       10   3    1   
     Matt Cassel          14    21   33   64  258        8   1    2   
     Matt Hasselbeck      34     6   11   55   43        4   0    0   
     Matt Ryan             1    23   31   74  299       10   3    0   
     Matt Schaub          18    20   31   65  266        9   1    0   
     Matthew Stafford     22    32   48   67  355        7   1    3   
     Michael Vick         16    29   56   52  317        6   2    4   
     Peyton Manning        7    19   26   73  253       10   2    0   
     Philip Rivers        19    24   33   73  231        7   1    0   
...                      ...   ...  ...  ...  ...      ...  ..  ...   
8    Mark Sanchez        272    28   54   52  283        5   1    1   
     Matt Cassel         271    20   30   67  218        7   1    1   
     Matt Hasselbeck     266    22   29   76  236        8   1    0   
     Matt Moore          276    11   19   58  131        7   1    0   
     Matt Ryan           256    22   29   76  262        9   3    0   
     Matthew Stafford    253    34   49   69  352        7   3    1   
     Michael Vick        264    21   35   60  191        6   1    0   
     Peyton Manning      255    22   30   73  305       10   3    0   
     Philip Rivers       277    18   34   53  154        5   0    0   
     Robert Griffin III  273    16   34   47  177        5   1    0   
     Russell Wilson      262    25   35   71  236        7   2    1   
     Ryan Mallett        282     1    3   33   17        6   0    0   
     Ryan Tannehill      281     2    5   40   18        4   0    0   
     Sam Bradford        274    22   30   73  205        7   1    1   
     Tim Tebow           283     0    0    0    0        0   0    0   
     Tom Brady           254    23   35   66  304        9   4    0   
     Tony Romo           260    36   62   58  437        7   1    4   
9    Aaron Rodgers       289    14   30   47  218        7   4    1   
     Andrew Luck         290    30   48   63  433        9   2    0   
     Andy Dalton         299    26   42   62  299        7   1    1   
     Cam Newton          291    13   23   57  201        9   1    0   
     Carson Palmer       288    39   61   64  414        7   4    3   
     Jay Cutler          293    19   26   73  229        9   3    0   
     Josh Freeman        297    18   30   60  247 

In [136]:
nfl_sql = df.to_sql(df, con=engine, index = df['Week'])

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [147]:
df.tail()

,Rk,Player,Pos,Week,Team,Opp,Comp,Att,Pct,Yds,Yds/Att,TD,Int,QB Rating,Att.1,Yds.1,Yds/Att.1,TD.1,Fantasy Points
295,296,Michael Vick,QB,9,PHI,NO,22,41,54,272,7,1,1,72,6,53,9,0,18.18
296,297,Josh Freeman,QB,9,TB,OAK,18,30,60,247,8,2,0,109,5,22,4,0,18.08
297,298,Tony Romo,QB,9,DAL,ATL,25,35,71,321,9,1,0,109,0,0,0,0,16.84
298,299,Andy Dalton,QB,9,CIN,DEN,26,42,62,299,7,1,1,81,2,16,8,0,15.56
299,300,Ryan Tannehill,QB,9,MIA,IND,22,38,58,290,8,1,0,91,1,-1,-1,0,15.50


In [2]:
position = ['QB', 'RB', 'WR', 'TE', 'DST', 'K']
week = ['1','2','3','4', '5','6','7','8','9','10','11','12','13','14','15','16','17']
season = ['2012', '2013','2014','2015','2016']


In [3]:
columns = ['player_id', 'name', 'team', 'vs', 'ffpts', 'patt', 'pcmp', 'pyds', 'ptds', 'pint', 'p2', 'ruatt', 'ruyds','rutds', 'ru2', 'rec', 'recyds', 'rectd', 'rec2', 'fum', 'fmbtd']
pos_week = pd.DataFrame(columns)

In [27]:
qb_1= pd.DataFrame()
rb_1= pd.DataFrame()
wr_1= pd.DataFrame()
te_1= pd.DataFrame()
k_1= pd.DataFrame()
qb_list = []
rb_list = []
wr_list = []
te_list = []
k_list = []
dst_list = []
parsed_qb_stats = []
parsed_rb_stats = []
parsed_wr_stats = []
parsed_te_stats = []
parsed_k_stats = []
qb_stats = []
team_list = []
vs_list = []

def player_name (pos):
    url = ('http://www.footballdb.com/fantasy-football/index.html?pos={}&yr=2012&wk=1&rules=1').format(pos)
    html = urlopen(url)
    bsobj= BeautifulSoup(html, 'lxml')
    for item in (bsobj.tbody({'a':'href'})):
        if pos == 'QB':
            if '.' not in item.get_text():
                qb_list.append(item.get_text())
            for item in bsobj.tbody.find_all({'td':'class'}):
                parsed_qb_stats.append(item.get_text().split())
                for idx in range(len(parsed_qb_stats)):
                    try:
                        stats.append(float(parsed_qb_stats[idx][0]))
                    except:
                        if len(parsed_qb_stats[idx]) > 1:
                            team_list.append(parsed_qb_stats[idx][4])
                            vs_list.append(parsed_qb_stats[idx+1][0].replace('@', ''))
            qb_1.team, qb_1.vs = team_list, vs_list
            stats = []
            team_list = []
            vs_list = []
            flat_list=[]
            remaining_columns = columns[4:]
            for num, col in enumerate(remaining_columns, start=2):
                flat_list = [item for sublist in parsed_qb_stats[num:][::19] for item in sublist]
                qb_1[col] = flat_list
        elif pos == 'RB':
            if '.' not in item.get_text():
                rb_list.append(item.get_text())
            for item in bsobj.tbody.find_all({'td':'class'}):
                parsed_rb_stats.append(item.get_text().split())
                for idx in range(len(parsed_rb_stats)):
                    try:
                        stats.append(float(parsed_rb_stats[idx][0]))
                    except:
                        if len(parsed_rb_stats[idx]) > 1:
                            team_list.append(parsed_rb_stats[idx][4])
                            vs_list.append(parsed_rb_stats[idx+1][0].replace('@', ''))
                rb_1.team, rb_1.vs = team_list, vs_list
                stats = []
                team_list = []
                vs_list = []
                flat_list = []
                remaining_columns = columns[4:]
                for num, col in enumerate(remaining_columns, start=2):
                    flat_list = [item for sublist in parsed_rb_stats[num:][::19] for item in sublist]
                    rb_1[col] = flat_list
        elif pos == 'TE':
            if '.' not in item.get_text():
                te_list.append(item.get_text())
            for item in bsobj.tbody.find_all({'td':'class'}):
                parsed_te_stats.append(item.get_text().split())
                for idx in range(len(parsed_te_stats)):
                    try:
                        stats.append(float(parsed_te_stats[idx][0]))
                    except:
                        if len(parsed_stats[idx]) > 1:
                            team_list.append(parsed_te_stats[idx][4])
                            vs_list.append(parsed_te_stats[idx+1][0].replace('@', ''))
                te_1.team, te_1.vs = team_list, vs_list
                stats = []
                team_list = []
                vs_list = []
                flat_list =[]
                remaining_columns = columns[4:]
                for num, col in enumerate(remaining_columns, start=2):
                    flat_list = [item for sublist in parsed_te_stats[num:][::19] for item in sublist]
                    te_1[col] = flat_list
        elif pos == 'WR':
            if '.' not in item.get_text():
                wr_list.append(item.get_text())
            for item in bsobj.tbody.find_all({'td':'class'}):
                parsed_wr_stats.append(item.get_text().split())
                for idx in range(len(parsed_wr__stats)):
                    try:
                        stats.append(float(parsed_wr_stats[idx][0]))
                    except:
                        if len(parsed_stats[idx]) > 1:
                            team_list.append(parsed_wr_stats[idx][4])
                            vs_list.append(parsed_wr_stats[idx+1][0].replace('@', ''))
                wr_1.team, wr_1.vs = team_list, vs_list
                stats = []
                team_list = []
                vs_list = []
                flat_list = []
                wr_1.team, wr_1.vs = team_list, vs_list
                remaining_columns = columns[4:]
                for num, col in enumerate(remaining_columns, start=2):
                    flat_list = [item for sublist in parsed_wr_stats[num:][::19] for item in sublist]
                    wr_1[col] = flat_list
        elif pos == 'K':
            if '.' not in item.get_text():
                k_list.append(item.getText())
            for item in bsobj.tbody.find_all({'td':'class'}):
                parsed_k_stats.append(item.get_text().split())
                for idx in range(len(parsed_k_stats)):
                    try:
                        stats.append(float(parsed_k_stats[idx][0]))
                    except:
                        if len(parsed_stats[idx]) > 1:
                            team_list.append(parsed_k_stats[idx][4])
                            vs_list.append(parsed_k_stats[idx+1][0].replace('@', ''))
                k_1.team, k_1.vs = team_list, vs_list
                stats = []
                team_list = []
                vs_list = []
                flat_list = []
                k_1.team, k_1.vs = team_list, vs_list
                remaining_columns = columns[4:]
                for num, col in enumerate(remaining_columns, start=2):
                    flat_list = [item for sublist in parsed_k_stats[num:][::19] for item in sublist]
                    k_1[col] = flat_list
        elif pos == 'DST':
            if  '.' not in item.get_text():
                dst_list.append(item.get_text())
# qb_1['name'] = qb_list
# qb_1['player_id'] = qb.index + 1

            

IndexError: list index out of range

In [128]:
def get_bsobj(pos):
    pos =[]
    url = ('http://www.footballdb.com/fantasy-football/index.html?pos={}&yr=2012&wk=1&rules=1').format(pos)
    html = urlopen(url)
    bsobj= BeautifulSoup(html, 'lxml')
    return bsobj

def player_name(bsobj):
    name =[]
    for item in (bsobj.tbody({'a':'href'})):
        if '.' not in item.get_text():
            name.append(item.get_text())
    return name, bsobj

def team_opponent_stats(bsobj):
    stats = []
    for item in bsobj.tbody.find_all({'td':'class'}):
            stats.append(item.get_text().split())
    team_list=[]
    vs_list = []
    for idx in range(len(stats)):
        try:
            stats.append(float(stats[idx][0]))
        except:
            if len(stats[idx]) > 1:
                team_list.append(stats[idx][4])
                vs_list.append(stats[idx+1][0].replace('@', ''))
    flat_list=[]
    remaining_columns = columns[4:]
    for num, col in enumerate(remaining_columns, start=2):
        flat_list = [item for sublist in stats[num:][::19] for item in sublist]
    return flat_list, team_list, vs

In [129]:
get_bsobj('qb')
player_name(bsobj)
qb_1 = pd.DataFrame(team_opponent(bsobj))

TypeError: 'list' object is not callable

In [126]:
qb_1= pd.DataFrame()
qb_1['name'] = player_name(bsobj)

In [127]:
qb_1

,name
0,"[Matt Ryan, Tony Romo, Robert Griffin III, Dre..."
1,"<!DOCTYPE html> <html lang=""en""> <head> <meta ..."


In [74]:
player_stats(parsed_stats)

[]

In [118]:
qb_1.team, qb_1.vs = team_list, vs_list

In [121]:
vs_list

[]

In [75]:
get_bsobj('qb')
player_name(bsobj)

(['Matt Ryan',
  'Tony Romo',
  'Robert Griffin III',
  'Drew Brees',
  'Mark Sanchez',
  'Aaron Rodgers',
  'Peyton Manning',
  'Blaine Gabbert',
  'Joe Flacco',
  'Jay Cutler',
  'Alex Smith',
  'Tom Brady',
  'Michael Vick',
  'Ben Roethlisberger',
  'Carson Palmer',
  'Ryan Fitzpatrick',
  'Matt Cassel',
  'Matt Schaub',
  'Philip Rivers',
  'Matthew Stafford',
  'Cam Newton',
  'Eli Manning',
  'Jake Locker',
  'Sam Bradford',
  'Josh Freeman',
  'Russell Wilson',
  'Andrew Luck',
  'Christian Ponder',
  'Kevin Kolb',
  'Andy Dalton',
  'John Skelton',
  'Ryan Tannehill',
  'Tim Tebow',
  'Tyrod Taylor',
  'Colin Kaepernick',
  'Matt Hasselbeck',
  'Bruce Gradkowski',
  'Brandon Weeden'],
 <!DOCTYPE html>
 <html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
 <title>2012 Fantasy Football Statistics | The Football Database</title>
 <meta content="View week 1

In [78]:
team_opponent(parsed_stats)

TypeError: team_opponent() missing 1 required positional argument: 'length'

In [76]:
parsed_stats(bsobj)

([['Matt', 'Ryan,', 'AtlM.', 'Ryan,', 'Atl'],
  ['@KC'],
  ['37.00'],
  ['31'],
  ['23'],
  ['299'],
  ['3'],
  ['0'],
  ['0'],
  ['3'],
  ['25'],
  ['1'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['Tony', 'Romo,', 'DalT.', 'Romo,', 'Dal'],
  ['@NYG'],
  ['29.00'],
  ['29'],
  ['22'],
  ['307'],
  ['3'],
  ['1'],
  ['0'],
  ['5'],
  ['12'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['Robert', 'Griffin', 'III,', 'WasR.', 'Griffin,', 'Was'],
  ['@NO'],
  ['28.00'],
  ['26'],
  ['19'],
  ['320'],
  ['2'],
  ['0'],
  ['0'],
  ['9'],
  ['42'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['Drew', 'Brees,', 'NOD.', 'Brees,', 'NO'],
  ['Was'],
  ['27.00'],
  ['52'],
  ['24'],
  ['339'],
  ['3'],
  ['2'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['0'],
  ['Mark', 'Sanchez,', 'NYJM.', 'Sanchez,', 'NYJ'],
  ['Buf'],
  ['26.00'],
  ['27'],
  ['19'],
  ['266'],
  

In [77]:
player_stats(parsed_stats)

[]

In [20]:
parsed_qb_stats

[['Matt', 'Ryan,', 'AtlM.', 'Ryan,', 'Atl'],
 ['@KC'],
 ['37.00'],
 ['31'],
 ['23'],
 ['299'],
 ['3'],
 ['0'],
 ['0'],
 ['3'],
 ['25'],
 ['1'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['Tony', 'Romo,', 'DalT.', 'Romo,', 'Dal'],
 ['@NYG'],
 ['29.00'],
 ['29'],
 ['22'],
 ['307'],
 ['3'],
 ['1'],
 ['0'],
 ['5'],
 ['12'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['Robert', 'Griffin', 'III,', 'WasR.', 'Griffin,', 'Was'],
 ['@NO'],
 ['28.00'],
 ['26'],
 ['19'],
 ['320'],
 ['2'],
 ['0'],
 ['0'],
 ['9'],
 ['42'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['Drew', 'Brees,', 'NOD.', 'Brees,', 'NO'],
 ['Was'],
 ['27.00'],
 ['52'],
 ['24'],
 ['339'],
 ['3'],
 ['2'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['Mark', 'Sanchez,', 'NYJM.', 'Sanchez,', 'NYJ'],
 ['Buf'],
 ['26.00'],
 ['27'],
 ['19'],
 ['266'],
 ['3'],
 ['1'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0'],
 ['0

In [14]:
k_list

['Blair Walsh',
 'Nate Kaeding',
 'Billy Cundiff',
 'Matt Bryant',
 'Justin Tucker',
 'David Akers',
 'Shayne Graham',
 'Nick Folk',
 'Greg Zuerlein',
 'Robbie Gould',
 'Stephen Hauschka',
 'Stephen Gostkowski',
 'Phil Dawson',
 'Connor Barth',
 'Josh Scobee',
 'Jason Hanson',
 'Sebastian Janikowski',
 'Jay Feely',
 'Shaun Suisham',
 'Mike Nugent',
 'Rob Bironas',
 'Ryan Succop',
 'Garrett Hartley',
 'Dan Bailey',
 'Lawrence Tynes',
 'Matt Prater',
 'Alex Henery',
 'Justin Medlock',
 'Rian Lindell',
 'Dan Carpenter',
 'Adam Vinatieri',
 'Mason Crosby',
 'Shane Lechler']

In [88]:
#Parse html to obtain list of QB's competeing in rleevant week 
qb_list= []
for item in (bsobj.tbody({'a':'href'})):
    if '.' not in item.get_text():
        qb_list.append(item.get_text())

qb_1['name'] = qb_list
qb_1['player_id'] = qb.index + 1

In [89]:
#Parse beautiful soup object to get only data needed in list format
parsed_stats = []
for item in bsobj.tbody.find_all({'td':'class'}):
    parsed_stats.append(item.get_text().split())

In [91]:
#Populate team and vs columns in qb dataframe by sorting parsed_list data
url = ('http://www.footballdb.com/fantasy-football/index.html?pos=qb&yr=2012&wk=1&rules=1')
html = urlopen(url)
bsobj= BeautifulSoup(html, 'lxml')
qb_list = []
parsed_qb_stats = []
team_list = []
vs_list = []
for item in (bsobj.tbody({'a':'href'})):
        if '.' not in item.get_text():
            qb_list.append(item.get_text())
        for item in bsobj.tbody.find_all({'td':'class'}):
            parsed_qb_stats.append(item.get_text().split())
for idx in range(len(parsed_stats)):
    try:
        stats.append(float(parsed_stats[idx][0]))
    except:
        if len(parsed_stats[idx]) > 1:
            team_list.append(parsed_stats[idx][4])
            vs_list.append(parsed_stats[idx+1][0].replace('@', ''))

qb_1.team, qb_1.vs = team_list, vs_list

remaining_columns = columns[4:]
for num, col in enumerate(remaining_columns, start=2):
    flat_list = [item for sublist in parsed_stats[num:][::19] for item in sublist]
    qb_1[col] = flat_list

2 ffpts
3 patt
4 pcmp
5 pyds
6 ptds
7 pint
8 p2
9 ruatt
10 ruyds
11 rutds
12 ru2
13 rec
14 recyds
15 rectd
16 rec2
17 fum
18 fmbtd


In [92]:
qb_1

,player_id,name,team,vs,ffpts,patt,pcmp,pyds,ptds,pint,...,ruatt,ruyds,rutds,ru2,rec,recyds,rectd,rec2,fum,fmbtd
0,1,Matt Ryan,Atl,KC,37.00,31,23,299,3,0,...,3,25,1,0,0,0,0,0,0,0
1,2,Tony Romo,Dal,NYG,29.00,29,22,307,3,1,...,5,12,0,0,0,0,0,0,0,0
2,3,Robert Griffin III,"Griffin,",NO,28.00,26,19,320,2,0,...,9,42,0,0,0,0,0,0,0,0
3,4,Drew Brees,NO,Was,27.00,52,24,339,3,2,...,0,0,0,0,0,0,0,0,0,0
4,5,Mark Sanchez,NYJ,Buf,26.00,27,19,266,3,1,...,0,0,0,0,0,0,0,0,0,0
5,6,Aaron Rodgers,GB,SF,26.00,44,30,303,2,1,...,5,27,0,0,0,0,0,0,0,0
6,7,Peyton Manning,Den,Pit,24.00,26,19,253,2,0,...,4,3,0,0,0,0,0,0,0,0
7,8,Blaine Gabbert,Jax,Min,23.00,39,23,260,2,0,...,5,6,0,0,0,0,0,0,1,0
8,9,Joe Flacco,Bal,Cin,23.00,29,21,299,2,0,...,2,-1,0,0,0,0,0,0,0,0
9,10,Jay Cutler,Chi,Ind,23.00,35,21,333,2,1,...,4,-4,0,0,0,0,0,0,0,0
